# Title basics : extraction only for movies and TV  movies

In [94]:
import pandas as pd
import datetime as dt
from datetime import datetime

In [95]:
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
# df_title_basics = pd.read_csv(url, sep='\t', compression='gzip',na_values=["\\N"], low_memory = False)

In [96]:
# df_title_basics.info(verbose=True, show_counts=True)

In [97]:
chunk_size = 1000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [98]:
cleaned_chunks = []
for chunk in iterator:
    # numeric_cols = chunk.select_dtypes(include=['number']).columns    #selectionner les colonne numerique
    # for col in numeric_cols:
    #     chunk[col].fillna(chunk[col].median(), inplace=True)      #remplacer les nans par le median

    if 'endYear' in chunk.columns:
        chunk.drop(columns=['endYear'], inplace=True)      #drop endYear
    chunk.drop(columns=['isAdult','originalTitle','runtimeMinutes'], inplace=True) #drop unused columns

    if 'genres' in chunk.columns:
        chunk.dropna(subset=['genres'], inplace=True)        #suppriner les colonnes ou le genre est nulls

    if 'runtimeMinutes' in chunk.columns:
        chunk['runtimeMinutes'] = pd.to_numeric(chunk['runtimeMinutes'], errors='coerce')      #convertir les valeurs de runtimeMinutes en numerique
        chunk.dropna(subset=['runtimeMinutes'], inplace=True)        #supprimer les lignes ou runtimeLinutes est null
        
    cleaned_chunks.append(chunk)

In [99]:
full_clean_basic_title = pd.concat(cleaned_chunks, ignore_index=True)      #concaterner mes chunks pour avoir un full dataset clean
print(full_clean_basic_title.info())
print(full_clean_basic_title.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10673658 entries, 0 to 10673657
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   tconst        object 
 1   titleType     object 
 2   primaryTitle  object 
 3   startYear     float64
 4   genres        object 
dtypes: float64(1), object(4)
memory usage: 407.2+ MB
None
          startYear
count  9.341503e+06
mean   2.006253e+03
std    1.988255e+01
min    1.874000e+03
25%    2.002000e+03
50%    2.013000e+03
75%    2.019000e+03
max    2.031000e+03


In [100]:
movie_types = ['movie', 'tvMovie']
df_title_basics_movies = full_clean_basic_title[full_clean_basic_title['titleType'].isin(movie_types)]
display(df_title_basics_movies)

df_title_basics_movies.info()

,tconst,titleType,primaryTitle,startYear,genres
8,tt0000009,movie,Miss Jerry,1894.0,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897.0,"Documentary,News,Sport"
569,tt0000574,movie,The Story of the Kelly Gang,1906.0,"Action,Adventure,Biography"
586,tt0000591,movie,The Prodigal Son,1907.0,Drama
609,tt0000615,movie,Robbery Under Arms,1907.0,Drama
...,...,...,...,...,...
10673577,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,2007.0,Documentary
10673582,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,2015.0,Drama
10673588,tt9916706,movie,Dankyavar Danka,2013.0,Comedy
10673598,tt9916730,movie,6 Gunn,2017.0,Drama


<class 'pandas.core.frame.DataFrame'>
Index: 754803 entries, 8 to 10673608
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tconst        754803 non-null  object 
 1   titleType     754803 non-null  object 
 2   primaryTitle  754801 non-null  object 
 3   startYear     656299 non-null  float64
 4   genres        754803 non-null  object 
dtypes: float64(1), object(4)
memory usage: 34.6+ MB


In [101]:
df_title_basics_movies['titleType'].unique()

array(['movie', 'tvMovie'], dtype=object)

In [102]:
#no need to keep title type ?
# subset_title_basics_movies.drop(['titleType'], axis=1, inplace=True)
df_title_basics_movies.reset_index(drop=True, inplace=True)
df_title_basics_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754803 entries, 0 to 754802
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tconst        754803 non-null  object 
 1   titleType     754803 non-null  object 
 2   primaryTitle  754801 non-null  object 
 3   startYear     656299 non-null  float64
 4   genres        754803 non-null  object 
dtypes: float64(1), object(4)
memory usage: 28.8+ MB


In [103]:
df_title_basics_movies.to_csv('df_title_basics_movies.csv', sep=';', index=False)
# !cp subset_moviessep.csv "drive/My Drive/"

### TMDB:On récupere les données complémentaires tmdb pour le productions_countries

Donwloaded csv from link : https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies
Then extraction of relevant columns


In [104]:
#https://www.nro.net/list-of-country-codes-in-the-lacnic-region/
country_code_list = ['AR','AW','BZ','BO','BR','CL','CO',
                     'CR','CU','DO','EC','SV','FK','GF',
                     'GT','GY','HT','HN','MX','AN','NI',
                     'PA','PY','PE','GS','SR','TT','UY','VE',
                     #'FR'
                     ]
###################################""
#
country_names = ['Argentina','Aruba','Bolivia','Brasil','Chile','Colombia',
                     'Costa Rica','Cuba','Dominican Republic','Ecuador','El Salvador',
                     'Guatemala',
                     'Guyana','Haiti','Honduras','Mexico',
                     'Nicaragua',
                     'Panama','Peru','Uruguay','Venezuela',
                     #'FR'
                     ]

In [105]:

url= "TMDB_movie_dataset_v11.csv"
df_tmdb = pd.read_csv(url,low_memory=False)
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118015 entries, 0 to 1118014
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1118015 non-null  int64  
 1   title                 1118002 non-null  object 
 2   vote_average          1118015 non-null  float64
 3   vote_count            1118015 non-null  int64  
 4   status                1118015 non-null  object 
 5   release_date          939457 non-null   object 
 6   revenue               1118015 non-null  int64  
 7   runtime               1118015 non-null  int64  
 8   adult                 1118015 non-null  bool   
 9   backdrop_path         298523 non-null   object 
 10  budget                1118015 non-null  int64  
 11  homepage              118817 non-null   object 
 12  imdb_id               597826 non-null   object 
 13  original_language     1118015 non-null  object 
 14  original_title        1118002 non-

In [106]:
df_tmdb.columns


Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [107]:
df_tmdb['production_countries'].unique()

array(['United Kingdom, United States of America',
       'United States of America, United Kingdom',
       'United States of America', ...,
       'Greece, Mexico, United States of America', 'Switzerland, Tunisia',
       'Bulgaria, Germany, United States of America'], dtype=object)

In [108]:
#define df_tmdb_latam
df_tmdb_latam = pd.DataFrame()
#columns we want to keep
columns_to_keep =['imdb_id', 'popularity', 'poster_path', 'production_countries']
df_tmdb = df_tmdb.loc[:,columns_to_keep]
#on ne garde que les pays  dans country_code_list
# et on identifie les pays pour lesquels on n'a aucun film

#drop na in df_tmdb['production_countries'] #remove rows with missing production country
df_tmdb.dropna(subset=['production_countries'], inplace=True)       
for country_name in  country_names:
  mask = df_tmdb['production_countries'].str.contains(country_name)
  #concat
  if (df_tmdb_latam.empty):
    df_tmdb_latam = df_tmdb[mask].copy()
  else:
    df_tmdb_latam = pd.concat([df_tmdb_latam, df_tmdb[mask]])
df_tmdb_latam.reset_index(drop=True, inplace= True)
df_tmdb_latam.info()


df_tmdb_latam.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31303 entries, 0 to 31302
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               20145 non-null  object 
 1   popularity            31303 non-null  float64
 2   poster_path           24969 non-null  object 
 3   production_countries  31303 non-null  object 
dtypes: float64(1), object(3)
memory usage: 978.3+ KB


,imdb_id,popularity,poster_path,production_countries
0,tt3011894,15.063,/pvSty5MCAxdj8LlAEXCDfM1YNOl.jpg,"Argentina, Spain"
1,tt1305806,17.242,/dkeAwfZzwL3WvToydE3CXiY80E0.jpg,"Argentina, Spain"
2,tt0168629,18.067,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,"Argentina, Denmark, Finland, France, Germany, ..."


In [109]:
df_tmdb_latam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31303 entries, 0 to 31302
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               20145 non-null  object 
 1   popularity            31303 non-null  float64
 2   poster_path           24969 non-null  object 
 3   production_countries  31303 non-null  object 
dtypes: float64(1), object(3)
memory usage: 978.3+ KB


In [110]:
df_tmdb_latam['production_countries'].unique()

array(['Argentina, Spain',
       'Argentina, Denmark, Finland, France, Germany, Iceland, Italy, Netherlands, Norway, Sweden, United Kingdom, United States of America',
       'Argentina, Chile, France, Germany, Peru, United Kingdom, United States of America',
       ..., 'Canada, France, Venezuela', 'France, Lebanon, Venezuela',
       'Portugal, Spain, Venezuela'], dtype=object)

In [111]:

#Merge
#inner join pour garder uniquement les ratings pour les films qui sont dans le df

df_merged_movies= pd.merge(df_title_basics_movies,df_tmdb_latam, how= 'inner',left_on='tconst',right_on= 'imdb_id')
#drop imdb_id
df_merged_movies.drop(columns=['imdb_id'],inplace=True)
df_merged_movies.reset_index(drop=True, inplace=True)
# del(df_tmdb_latam)
# del(df_title_basics_movies)
#format year as integer
df_merged_movies.dropna(subset=['startYear'],inplace=True)
df_merged_movies['startYear'] = df_merged_movies['startYear'].astype(int)

display(df_merged_movies.head(3))
display(df_merged_movies.info())

# df_merged_movies.drop(columns= ['titleId','region'],inplace=True)
# df_merged_movies.info()

# #garder le titre en FR (title) s'il n'est pas null, sinon primary title
# df_merged_movies['title'] = df_merged_movies['title'].fillna(df_merged_movies['primaryTitle'])
# df_merged_movies.info()


,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico
1,tt0007638,movie,El amor que huye,1917,Drama,1.400,NaN,Mexico
2,tt0007646,movie,El apóstol,1917,"Animation,Comedy,Drama",0.840,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Argentina


<class 'pandas.core.frame.DataFrame'>
Index: 14933 entries, 0 to 15226
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                14933 non-null  object 
 1   titleType             14933 non-null  object 
 2   primaryTitle          14933 non-null  object 
 3   startYear             14933 non-null  int64  
 4   genres                14933 non-null  object 
 5   popularity            14933 non-null  float64
 6   poster_path           12810 non-null  object 
 7   production_countries  14933 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.0+ MB


None

## Title akas (traduction des titres en Espagnol ? plutot qu'en français? ou les deux)

In [112]:
url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

chunk_size = 10000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

cleaned_chunks = []
for chunk in iterator:

    if 'attributes' in chunk.columns:
        chunk.drop(columns=['attributes'], inplace=True)      #drop attributes

    if 'types' in chunk.columns:
        #Peggi
        chunk.drop(chunk[chunk['types']!='imdbDisplay'].index, inplace=True)      #drop rows with types different from imdbDisplay
         ##
        chunk.drop(columns=['types'], inplace=True)      #drop types

    if 'isOriginalTitle' in chunk.columns:
        chunk.drop(chunk[chunk['isOriginalTitle'] == 1].index, inplace=True)      #drop rows with isOriginalTitle

    if 'region' in chunk.columns:
        #keep all raws whose region is in country_code_list
        region_list = ['AR','AW','BZ','BO','BR','CL','CO',
                      'CR','CU','DO','EC','SV','FK','GF',
                      'GT','GY','HT','HN','MX','AN','NI',
                      'PA','PY','PE','GS','SR','TT','UY','VE']
        chunk = chunk.loc[chunk['region'].isin(region_list)]

    cleaned_chunks.append(chunk)

#on concatene
full_clean_title_akas = pd.concat(cleaned_chunks, ignore_index=True)

In [113]:
full_clean_title_akas['region'] = full_clean_title_akas['region'].astype(str)
full_clean_title_akas = full_clean_title_akas.drop(columns=['ordering','language','isOriginalTitle'])
full_clean_title_akas.head()

,titleId,title,region
0,tt0000012,A Chegada de um Trem à Estação,BR
1,tt0000014,O Regador Regado,BR
2,tt0000028,A Pesca do Peixe Dourado,BR
3,tt0000029,O Almoço do Bebê,BR
4,tt0000033,A Acrobacia,BR


In [114]:
#Merge
#inner join pour garder tous les films qui sont dans le df
df_merged_movies_2= pd.merge(df_merged_movies,full_clean_title_akas, how= 'left',left_on='tconst',right_on= 'titleId')
#drop title_id
df_merged_movies_2.drop(columns=['titleId'],inplace=True)
df_merged_movies_2.reset_index(drop=True, inplace=True)
del(full_clean_title_akas)

display(df_merged_movies_2.head(3))
display(df_merged_movies_2.info())

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,MX
2,tt0007638,movie,El amor que huye,1917,Drama,1.400,NaN,Mexico,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19773 entries, 0 to 19772
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                19773 non-null  object 
 1   titleType             19773 non-null  object 
 2   primaryTitle          19773 non-null  object 
 3   startYear             19773 non-null  int64  
 4   genres                19773 non-null  object 
 5   popularity            19773 non-null  float64
 6   poster_path           17491 non-null  object 
 7   production_countries  19773 non-null  object 
 8   title                 18350 non-null  object 
 9   region                18350 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.5+ MB


None

## On récupère title principals (actors and actresses)

In [115]:
url = 'https://datasets.imdbws.com/title.principals.tsv.gz'
chunk_size = 10000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [116]:
cleaned_chunks = []
for chunk in iterator:
    # numeric_cols = chunk.select_dtypes(include=['number']).columns    #selectionner les colonne numerique
    if 'job' in chunk.columns:
        chunk.drop(columns=['job'], inplace=True)      #drop job

    #on supprime la colonne knownForTitles
    chunk = chunk.loc[(chunk['category'] =='actor') | (chunk['category']== 'actress') | (chunk['category']== 'director')].copy()
    chunk.drop(columns=['ordering','category','characters'], inplace = True)
    #############
    cleaned_chunks.append(chunk)
    del(chunk)

In [117]:

subset_actors_directors = pd.concat(cleaned_chunks, ignore_index=True)      #concaterner mes chunks pour avoir un full dataset actors clean

#On supprime les doublons
subset_actors_directors= subset_actors_directors.drop_duplicates(subset=None, keep="first", inplace=False)
subset_actors_directors.reset_index(drop=True, inplace=True)
#print(df_title_ppals_actress.duplicated().value_counts())
subset_actors_directors.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42778849 entries, 0 to 42778848
Data columns (total 2 columns):
 #   Column  Non-Null Count     Dtype 
---  ------  --------------     ----- 
 0   tconst  42778849 non-null  object
 1   nconst  42778849 non-null  object
dtypes: object(2)
memory usage: 652.8+ MB


In [118]:
# inner join? it seems enough as we only want movies with actors ;p
df_merged_movies_3 = pd.merge(df_merged_movies_2,subset_actors_directors, how= 'inner',on='tconst')

# del(subset_actors_directors)

df_merged_movies_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176822 entries, 0 to 176821
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                176822 non-null  object 
 1   titleType             176822 non-null  object 
 2   primaryTitle          176822 non-null  object 
 3   startYear             176822 non-null  int64  
 4   genres                176822 non-null  object 
 5   popularity            176822 non-null  float64
 6   poster_path           158707 non-null  object 
 7   production_countries  176822 non-null  object 
 8   title                 167357 non-null  object 
 9   region                167357 non-null  object 
 10  nconst                176822 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 14.8+ MB


## Name basics

In [152]:
url = 'https://datasets.imdbws.com/name.basics.tsv.gz'
df_name_basicschunk_size = 1000
iterator = pd.read_csv(url, sep='\t', compression='gzip', iterator=True, chunksize=chunk_size, na_values=["\\N"])

In [153]:
cleaned_chunks = []

for chunk in iterator:

    # if 'deathYear' in chunk.columns:
    #     chunk['deathYear'].fillna(0, inplace=True)

    if 'primaryName' in chunk.columns:
        chunk.dropna(subset=['primaryName'], inplace=True)

    if 'primaryProfession' in chunk.columns:
        chunk.dropna(subset=['primaryProfession'], inplace=True)
    chunk.drop(['knownForTitles','deathYear','birthYear'], axis=1, inplace=True)
    # chunk['birthYear'] = pd.to_datetime(chunk['birthYear'], format='%Y', errors='coerce')

    cleaned_chunks.append(chunk)

In [154]:
df_name_basics = pd.concat(cleaned_chunks, ignore_index=True)
display(df_name_basics.head(10))
print(df_name_basics.info())
print(df_name_basics.describe())

,nconst,primaryName,primaryProfession
0,nm0000001,Fred Astaire,"actor,miscellaneous,producer"
1,nm0000002,Lauren Bacall,"actress,soundtrack,archive_footage"
2,nm0000003,Brigitte Bardot,"actress,music_department,producer"
3,nm0000004,John Belushi,"actor,writer,music_department"
4,nm0000005,Ingmar Bergman,"writer,director,actor"
5,nm0000006,Ingrid Bergman,"actress,producer,soundtrack"
6,nm0000007,Humphrey Bogart,"actor,producer,miscellaneous"
7,nm0000008,Marlon Brando,"actor,director,writer"
8,nm0000009,Richard Burton,"actor,producer,director"
9,nm0000010,James Cagney,"actor,director,producer"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11195640 entries, 0 to 11195639
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   primaryProfession  object
dtypes: object(3)
memory usage: 256.2+ MB
None
           nconst    primaryName primaryProfession
count    11195640       11195640          11195640
unique   11195640        8818574             23046
top     nm9993718  Michael Smith             actor
freq            1            344           2465416


In [123]:
# full_clean_basics_name['birthYear'] = full_clean_basics_name['birthYear'].dt.year
# full_clean_basics_name['birthYear'] = full_clean_basics_name['birthYear'].astype('Int64')
# full_clean_basics_name['deathYear'] = full_clean_basics_name['deathYear'].astype('Int64')
# full_clean_basics_name.info()

In [124]:
# full_clean_basics_name['birthYear'].fillna(0, inplace=True)

In [ ]:
# actors = ["actor", "actress"]#, "director","writer"]
# subset_professions_actors = full_clean_basics_name[full_clean_basics_name["primaryProfession"].apply(lambda x: any(profession in x.split(',') for profession in professions))]
# display(subset_professions_actors)

In [155]:
# rename column 'nconst' to 'a_id' pour actors id
# df_merged_movies_4 = df_merged_movies_3.rename(columns={'nconst': 'a_id'})
# jointure  name_basics avec df_merged_series
df_merged_movies_4 = pd.merge(df_merged_movies_3,df_name_basics, how= 'inner',on='nconst') #left_on='a_id',right_on='nconst')

# df_merged_movies_4.drop(columns = 'nconst',inplace= True)
# rename column  'primaryName' to actor name
# df_merged_movies_4 = df_merged_movies_4.rename(columns={'primaryName': 'actorName','primaryName': 'actorName'})

df_merged_movies_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175390 entries, 0 to 175389
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                175390 non-null  object 
 1   titleType             175390 non-null  object 
 2   primaryTitle          175390 non-null  object 
 3   startYear             175390 non-null  int64  
 4   genres                175390 non-null  object 
 5   popularity            175390 non-null  float64
 6   poster_path           157521 non-null  object 
 7   production_countries  175390 non-null  object 
 8   title                 166237 non-null  object 
 9   region                166237 non-null  object 
 10  nconst                175390 non-null  object 
 11  primaryName           175390 non-null  object 
 12  primaryProfession     175390 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 17.4+ MB


In [156]:
df_merged_movies_4.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,primaryName,primaryProfession
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0220230,Mimí Derba,"actress,producer,director"
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0846016,Julio Taboada,actor
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0704415,Salvador Quiroz,"actor,miscellaneous,archive_footage"
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0749618,Emilia Ruiz del Castillo,actress
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0035999,Salvador Arnaldo,actor


In [157]:
df_merged_movies_4 = df_merged_movies_4.drop_duplicates(ignore_index=True)
df_merged_movies_4.head()

,tconst,titleType,primaryTitle,startYear,genres,popularity,poster_path,production_countries,title,region,nconst,primaryName,primaryProfession
0,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0220230,Mimí Derba,"actress,producer,director"
1,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0846016,Julio Taboada,actor
2,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0704415,Salvador Quiroz,"actor,miscellaneous,archive_footage"
3,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0749618,Emilia Ruiz del Castillo,actress
4,tt0007627,movie,Alma de sacrificio,1917,"Drama,Romance",1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Mexico,Alma de sacrificio,EC,nm0035999,Salvador Arnaldo,actor


In [158]:
df_merged_movies_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162048 entries, 0 to 162047
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                162048 non-null  object 
 1   titleType             162048 non-null  object 
 2   primaryTitle          162048 non-null  object 
 3   startYear             162048 non-null  int64  
 4   genres                162048 non-null  object 
 5   popularity            162048 non-null  float64
 6   poster_path           144513 non-null  object 
 7   production_countries  162048 non-null  object 
 8   title                 153383 non-null  object 
 9   region                153383 non-null  object 
 10  nconst                162048 non-null  object 
 11  primaryName           162048 non-null  object 
 12  primaryProfession     162048 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 16.1+ MB


In [159]:
#regrouper tous les noms  par film
columns_to_keep = ['tconst',#'title',
                   'primaryTitle','startYear','genres',
                #    'actorName','writerName',
                    'primaryName',
                   #'region',
                   'production_countries','popularity','poster_path']
df_grouped = df_merged_movies_4.groupby(columns_to_keep,as_index=False)['primaryName'].apply(lambda x: ', '.join(x) )
# df_grouped = df_merged_movies_4.groupby(columns_to_keep,as_index=False)['primaryName'].sum().reset_index(drop=True)
# del(df_merged_movies)
df_grouped.head()

,tconst,primaryTitle,startYear,genres,production_countries,popularity,poster_path,primaryName
0,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,"Emilia Ruiz del Castillo, Emilia Ruiz del Cast..."
1,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,"Joaquín Coss, Joaquín Coss"
2,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,"Julio Taboada, Julio Taboada"
3,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,"Manuel Arvide, Manuel Arvide"
4,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,"Manuel Campa Siliceo, Manuel Campa Siliceo"


In [160]:

def remove_duplicates(s):
    # Diviser la chaîne en sous-chaînes, enlever les doublons et les recombiner
    unique_parts = set(s.split(', '))
    return ', '.join(unique_parts)

# Appliquer la fonction à la colonne primaryName
df_grouped['primaryName'] = df_grouped['primaryName'].apply(remove_duplicates)
df_grouped.head()

,tconst,primaryTitle,startYear,genres,production_countries,popularity,poster_path,primaryName
0,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Emilia Ruiz del Castillo
1,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Joaquín Coss
2,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Julio Taboada
3,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Manuel Arvide
4,tt0007627,Alma de sacrificio,1917,"Drama,Romance",Mexico,1.151,/sAwbE4IhwnTdRqODTDI6PCFWdZ3.jpg,Manuel Campa Siliceo


In [161]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103149 entries, 0 to 103148
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   tconst                103149 non-null  object 
 1   primaryTitle          103149 non-null  object 
 2   startYear             103149 non-null  int64  
 3   genres                103149 non-null  object 
 4   production_countries  103149 non-null  object 
 5   popularity            103149 non-null  float64
 6   poster_path           103149 non-null  object 
 7   primaryName           103149 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 6.3+ MB


In [ ]:
# #On remplace les virgules par de espaces dans la colonne genre pour uniformiser avec les autres colonnes
# df_grouped['genres'] = df_grouped['genres'].str.replace(',',' ')
# df_grouped.info()
# df_grouped.head()


## Title ratings

In [162]:
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
df_title_ratings = pd.read_csv(url_ratings, sep='\t', compression='gzip', na_values=["\\N"])

#inner join pour garder uniquement les ratings pour les films qui sont dans le df
df_merged_popular = pd.merge(df_grouped,df_title_ratings, how= 'inner',on='tconst')

# del(df_title_ratings)
# del(df_grouped)
display(df_merged_popular.head(3))
display(df_merged_popular.info())

,tconst,primaryTitle,startYear,genres,production_countries,popularity,poster_path,primaryName,averageRating,numVotes
0,tt0007646,El apóstol,1917,"Animation,Comedy,Drama",Argentina,0.84,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,Quirino Cristiani,7.1,77
1,tt0007958,Peach Blossom,1917,Drama,Argentina,0.60,/5IwfGMcwBXhtGDRWJo5X5mqQPu5.jpg,Argentino Gómez,6.1,31
2,tt0007958,Peach Blossom,1917,Drama,Argentina,0.60,/5IwfGMcwBXhtGDRWJo5X5mqQPu5.jpg,Carlos Gardel,6.1,31


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88259 entries, 0 to 88258
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                88259 non-null  object 
 1   primaryTitle          88259 non-null  object 
 2   startYear             88259 non-null  int64  
 3   genres                88259 non-null  object 
 4   production_countries  88259 non-null  object 
 5   popularity            88259 non-null  float64
 6   poster_path           88259 non-null  object 
 7   primaryName           88259 non-null  object 
 8   averageRating         88259 non-null  float64
 9   numVotes              88259 non-null  int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 6.7+ MB


None

In [169]:
#regrouper tous les noms  par film
columns_to_keep = ['tconst',#'title',
                   'primaryTitle','startYear','genres',
                #    'actorName','writerName',
                    # 'primaryName',
                   #'region',
                   'production_countries','popularity','poster_path',
                   'averageRating','numVotes']
df_grouped_2 = df_merged_popular.groupby(columns_to_keep,as_index=False)['primaryName'].apply(lambda x: ', '.join(x) )
# Appliquer la fonction à la colonne primaryName
df_grouped_2['primaryName'] = df_grouped_2['primaryName'].apply(remove_duplicates)
display(df_grouped_2.head())
df_grouped_2.info()


,tconst,primaryTitle,startYear,genres,production_countries,popularity,poster_path,averageRating,numVotes,primaryName
0,tt0007646,El apóstol,1917,"Animation,Comedy,Drama",Argentina,0.84,/xOyr2LTVbHiAiHY8mLNGqt68Llg.jpg,7.1,77,Quirino Cristiani
1,tt0007958,Peach Blossom,1917,Drama,Argentina,0.60,/5IwfGMcwBXhtGDRWJo5X5mqQPu5.jpg,6.1,31,"Ilde Pirovano, Argentino Gómez, Mariano Gale, ..."
2,tt0008662,Tepeyac,1917,Drama,Mexico,0.60,/eccHRe1Tt1sv1pMcnPNFOZNsZS.jpg,5.8,70,"Pilar L. Cotta, Roberto Arroyo Carrillo, Carlo..."
3,tt0008807,The Last Indian Attack,1917,"Documentary,History,War",Argentina,0.60,/nkz2QoeQkn8EwvnQmscokUm9PE6.jpg,6.5,111,"Rosa Volpe, Salvador López, Alcides Greca, Mar..."
4,tt0009619,Sin dejar rastros,1918,Animation,Argentina,0.60,/7IiiMRXz0aYxqpKUxHGuediqqGb.jpg,5.9,25,Quirino Cristiani


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9851 entries, 0 to 9850
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tconst                9851 non-null   object 
 1   primaryTitle          9851 non-null   object 
 2   startYear             9851 non-null   int64  
 3   genres                9851 non-null   object 
 4   production_countries  9851 non-null   object 
 5   popularity            9851 non-null   float64
 6   poster_path           9851 non-null   object 
 7   averageRating         9851 non-null   float64
 8   numVotes              9851 non-null   int64  
 9   primaryName           9851 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 769.7+ KB


## Ecriture du df trié par popularité dans un fichier csv 

In [170]:
df_merged_popular = df_merged_popular.sort_values(by=['averageRating','numVotes'], ascending=False)

df_merged_popular.to_csv("grouped_movies_latam_ML.csv",index=False,sep =';')